In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

import unicodecsv as csv

In [2]:
def crawling(url, category, debug = False):
    driver = webdriver.Chrome()
    driver.get(url)

    master = [['Category', 'Title', 'Description', 'PostedTime', 'NumberOfComments', 'NumberOfBookmarks']]

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        elements = driver.find_elements_by_class_name('mlist')

        for element in elements:
            temp = [category]

            #------ find title of deal
            try:
                title = element.find_element_by_tag_name('h1').find_element_by_tag_name('a').text
            except:
                pass

            try:
                title = element.find_element_by_tag_name('h2').find_element_by_tag_name('a').text
            except:
                pass

            temp.append(title)


            #------ find description of deal
            des = ''
            des_elements = element.find_element_by_class_name('mbody').find_elements_by_tag_name('li')
            
            if len(des_elements) == 0:
                des_elements = element.find_element_by_class_name('mbody').find_elements_by_tag_name('p')
            
            if len(des_elements) == 0:
                des = element.find_element_by_class_name('mbody').find_element_by_tag_name('div').text
                
            if len(des_elements) == 0 and len(des) == 0:
                print '===DESCRIPTION NOT FOUND'
                print 'deal skipped'
                continue
            
            for des_element in des_elements:
                des += des_element.text
                des += '\n '
                    
            temp.append(des)
            


            #------ find time the deal was posted
            try:
                time = element.find_element_by_class_name('date').text
                temp.append(time[0:-4]) #strip away 'Posted' and 'ago'
            except Exception as e:
                print '===TIME NOT FOUND'
                print '===deal skipped'
                continue


            #------ find number of comments for the deal
            try:
                num_comments = element.find_element_by_class_name('pl_btn').find_element_by_tag_name('a').text
                # If there are more than 1 comment, "Comment" is written as "Comments"
                if len(num_comments) > 11:
                    temp.append(num_comments[10:-1])
                else:
                    temp.append(num_comments[9:-1])
            except Exception as e:
                temp.append('0')
                pass


            #------ find number of comments for the deal
            try:
                num_comments = element.find_element_by_class_name('fav_btn').find_element_by_tag_name('em').text
                temp.append(num_comments[1:-1])
            except Exception as e:
                temp.append('0')
                pass

            #------ append to master list
            master.append(temp)


        try:
            load = driver.find_element_by_xpath("//a[@title='Next']")

            # See if the last page has been reached
            page_num = driver.find_element_by_class_name('pages').find_element_by_class_name('current').text
            
            if page_num == u'100':
                print 'Last page reached'
                break
            else:
                load.click()          
        except:
            print "===Can't go to the next page"
            break 
            
        if debug == True:
            break
            
    return master

In [3]:
def saveCSV(filename, data):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data)

# Crawl data

In [ ]:
all_categories = ['Clothing', 'Beauty', 'Nutrition', 'Baby', 'Home', 'Electronics', 'Travel', 'Finance']
all_urls = ['http://www.dealmoon.com/Clothing-Jewelry-Bags',
           'http://www.dealmoon.com/Beauty',
           'http://www.dealmoon.com/Nutrition-Supplements',
           'http://www.dealmoon.com/Baby',
           'http://www.dealmoon.com/Home-Garden',
           'http://www.dealmoon.com/Electronics',
           'http://www.dealmoon.com/Travel',
           'http://www.dealmoon.com/Finance']

for i in range(len(all_categories)):
    data = crawling(all_urls[i], all_categories[i])
    print len(data)
    filename = all_categories[i] + '.csv'
    saveCSV(filename, data)